In [1]:
!pip install rpy2==3.5.1

In [2]:
%load_ext rpy2.ipython

Unable to determine R home: [WinError 2] The system cannot find the file specified
C:\Anaconda\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
%%R
# data preprocessing
library(tidyverse)
library(lubridate)
# data exploration
library(summarytools) # for user-friendly html summaries of data
library(ggmap) # for plotting data on a map
# for meta-ml
library(tidymodels)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.3.6      v purrr   0.3.5 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.1      v stringr 1.4.1 
v readr   2.1.3      v forcats 0.5.2 
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


R[write to console]: Loading required package: timechange

R[write to console]: 
Attaching package: 'lubridate'


R[write to console]: The following objects are masked from 'package:base':

    date, intersect, setdiff, union


R[write to console]: 
Attaching package: 'summarytools'


R[write to console]: The following object is masked from 'package:tibble':

    view




i Google's Terms of Service: <]8;;https://mapsplatform.google.comhttps://mapsplatform.google.com]8;;>
i Please cite ggmap if you use it! Use `citation("ggmap")` for details.


R[write to console]: -- Attaching packages ------------------------------------------------------------------------------ tidymodels 1.0.0 --

R[write to console]: v broom        1.0.1     v rsample      1.1.1
v dials        1.1.0     v tune         1.0.1
v infer        1.0.4     v workflows    1.1.2
v modeldata    1.0.1     v workflowsets 1.0.0
v parsnip      1.0.3     v yardstick    1.1.0
v recipes      1.0.3     

R[write to console]: -- Conflicts --------------------------------------------------------------------------------- tidymodels_conflicts() --
x scales::discard()    masks purrr::discard()
x dplyr::filter()      masks stats::filter()
x recipes::fixed()     masks stringr::fixed()
x dplyr::lag()         masks stats::lag()
x yardstick::spec()    masks readr::spec()
x recipes::step()      masks stats::step()
x summarytools::view() masks tibble::view()
* Learn how to get started at https://www.tidymodels.org/start/



In [4]:
%%R
# let's set some global options
options(dplyr.width = Inf) # show all columns when printing to console
theme_set(theme_minimal()) # select a lightweight ggplot theme for cleaner plotting
set.seed(2023) # in the AC, you'll be required to set a fixed random seed to make your work reproducible

In [5]:
%%R
# Start reading data
classification <- read_csv( 'classification.csv' )

customers <- read_csv(
  'customers.csv',
  col_types = cols(
    Item_Position = col_double() 
  )
)#change from character like "005230" to numeric number "5230"

sales_orders <- read_csv( 'sales_orders.csv')

sales_orders_header <- read_csv('sales_orders_header.csv' ) %>% 
  mutate(
    Release_Date = as_date(Release_Date) %>% as.character(),
    Creation_Date = as_date(Creation_Date) %>% as.character()
  )
#which(is.na(sales_orders_header$Release_Date)) #look if as_data is not run, exact time00:00:00 is added.
# 150048 header$Sales_Order, compared to 150064 sales_order$Sales_Order

business_units <- read_csv('business_units.csv') %>% 
  select( -YHKOKRS ) 
# YHKOKRS, all elements are 5180, useless

service_map <- read_csv('service_map.csv') 
#MATKL_service 29 distict values, compared to sales_orders$Material_Class with 329 distict values

R[write to console]: 
R[write to console]: indexing classification.csv [=============================] 139.45MB/s, eta:  0s
                                                                                                                   
R[write to console]: 


Rows: 2561 Columns: 3
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (1): Customer_ID
dbl (2): Reseller, Test_set_id

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: 
R[write to console]: indexing customers.csv [====--------------------------------] 2.15GB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing customers.csv [==================================] 611.32MB/s, eta:  0s
                                                                                                                   
R[write to console]: 
R[write to console]: 
R[write to console]: indexing sales_orders.csv [---------------------------------] 2.15GB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders.csv [=============------------------] 551.33MB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders.csv [===============----------------] 550.05MB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders.csv [==================-------------] 360.99MB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders.csv [====================-----------] 370.84MB/s, eta

Rows: 1248889 Columns: 7
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (3): Sales_Order, Material_Code, Cost_Center
dbl (4): Item_Position, Num_Items, Material_Class, Net_Value

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: 
R[write to console]: indexing sales_orders_header.csv [=-------------------------] 2.15GB/s, eta:  0s
R[write to console]: 
R[write to console]: indexing sales_orders_header.csv [========================] 469.15MB/s, eta:  0s
                                                                                                                   
R[write to console]: 


Rows: 150058 Columns: 8
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr  (6): Sales_Organization, Sales_Order, Creator, Document_Type, Release_D...
dbl  (1): Net_Value
dttm (1): Creation_Date

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: 
R[write to console]: indexing business_units.csv [=============================] 600.53MB/s, eta:  0s
                                                                                                                   
R[write to console]: 


Rows: 126 Columns: 3
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
chr (2): Cost_Center, Business_Unit
dbl (1): YHKOKRS

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: 
R[write to console]: indexing service_map.csv [=================================] 11.16MB/s, eta:  0s
                                                                                                                   
R[write to console]: 


Rows: 29 Columns: 1
-- Column specification ------------------------------------------------------------------------------------------------
Delimiter: ","
dbl (1): MATKL_service

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
%%R
#Start joining tibble data
cla_cus <- left_join(classification, customers, by = c('Customer_ID' = 'Customer_ID'))

#join sales order
cla_cus_sal_inner <- inner_join(cla_cus, sales_orders, by = c('Sales_Order' = 'Sales_Order','Item_Position' = 'Item_Position'))

sales_orders_rest <- anti_join(sales_orders, cla_cus, by = c('Sales_Order' = 'Sales_Order','Item_Position' = 'Item_Position')  )
sales_orders_rest <- sales_orders_rest %>% mutate(Item_Position = 0 )
cla_cus_sal_rest <- inner_join(cla_cus, sales_orders_rest, by = c('Sales_Order' = 'Sales_Order','Item_Position' = 'Item_Position'))

cla_cus_sal <- bind_rows(cla_cus_sal_inner, cla_cus_sal_rest)
#cla_cus_sal %>% dfSummary %>% view  #cla_cus_sal$Test_set_id has 512 distinct values, which is same as classification.csv

#join sales order header
cla_cus_sal_head <- left_join(cla_cus_sal, sales_orders_header, by = c('Sales_Order' = 'Sales_Order'))

#join business unit
cla_cus_sal_head_busi <- left_join(cla_cus_sal_head, business_units, by = c('Cost_Center' = 'Cost_Center'))
all_merged <- cla_cus_sal_head_busi

#join service maps
all_merged = all_merged %>%
  mutate(
    MATKL = case_when(
      Material_Class %in% service_map$MATKL_service ~ 1,
      TRUE ~ 0 # TRUE is equivalent to an “else” statement.
    )
  )
#all_merged %>% dfSummary %>% view

In [7]:
%%R
print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(all_merged)) { # for-loop over columns
  print(paste0(names(all_merged)[i], ": ", sum(is.na(all_merged[, i])), "/", nrow(all_merged)))
}
print("[INFO] Glimpse:")
glimpse(all_merged)

#join and merge is finished above
#####################################################################################
df <- all_merged

[1] "[INFO] Number of NA values in each column:"
[1] "Customer_ID: 0/1409894"
[1] "Reseller: 305813/1409894"
[1] "Test_set_id: 1104081/1409894"
[1] "Sales_Order: 0/1409894"
[1] "Item_Position: 0/1409894"
[1] "Type: 0/1409894"
[1] "Num_Items: 0/1409894"
[1] "Material_Code: 0/1409894"
[1] "Material_Class: 2/1409894"
[1] "Cost_Center: 0/1409894"
[1] "Net_Value.x: 0/1409894"
[1] "Sales_Organization: 43/1409894"
[1] "Creation_Date: 43/1409894"
[1] "Creator: 43/1409894"
[1] "Document_Type: 43/1409894"
[1] "Release_Date: 3683/1409894"
[1] "Delivery: 43/1409894"
[1] "Net_Value.y: 43/1409894"
[1] "Business_Unit: 0/1409894"
[1] "MATKL: 0/1409894"
[1] "[INFO] Glimpse:"
Rows: 1,409,894
Columns: 20
$ Customer_ID        <chr> "0x519277E9B1C951A10E3C688949F69A00C623ADC26AADD09B~
$ Reseller           <dbl> 0, 0, 0, 0, 0, NA, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,~
$ Test_set_id        <dbl> NA, NA, NA, NA, NA, 137, NA, NA, NA, NA, NA, NA, NA~
$ Sales_Order        <chr> "0xE52DCC1B2EF98A9D36D30F9C91514285612

In [8]:
%%R
###Handling binary features
#  "Type": Replace string values "SOP,STP" with binary values
df = df %>%
  mutate(
    Type_SOTP = case_when(
      Type %in% c("SOP") ~ 1,
      Type %in% c("STP") ~ 0,
    )
  )
df = select (df,-c(Type))

#  "Sales_Organization": Replace string values "A,B " with binary values
df = df %>%
  mutate(
    Organization = case_when(
      Sales_Organization %in% c("A") ~ 1,
      Sales_Organization %in% c("B") ~ 0,
    )
  )
df = select (df,-c(Sales_Organization))

#  "Creator": "0x2500D9>69.7%"
df = df %>%
  mutate(
    Creator_Binary = case_when(
      Creator == "0x2500D9E578134DB667D820EA3100F880DAF13E408E6761CEE69DBFC3EAF833B4" ~ 1,
      TRUE ~ 0
    )
  )

In [9]:
%%R
###Handling date
#  "Creation_Date" and "Release_Date"
# Compare them with each other first
df = df %>%
  mutate(crea_relea = case_when(
    Creation_Date==Release_Date ~ 1,
    TRUE ~ 0
  ))
#df$crea_relea %>% dfSummary %>% view
df <- df %>% select(-crea_relea)
# Creation_Date 
#year
df = df %>%
  mutate(Creation_Year = case_when(
    !is.na(Creation_Date) ~ Creation_Date %>%
      substr(nchar(Creation_Date) - 9, nchar(Creation_Date)-6 ) %>%
      as.numeric()
  ))
df = df %>% mutate(Creation_Year = (Creation_Year-2020)* 365 )
#month
df = df %>%
  mutate(Creation_Month = case_when(
    !is.na(Creation_Date) ~ Creation_Date %>%
      substr(nchar(Creation_Date) - 4, nchar(Creation_Date)-3 ) %>%
      as.numeric()
  ))
df = df %>% mutate(Creation_Month = ( Creation_Month-1)* 30 )
#day
df = df %>%
  mutate(Creation_Day = case_when(
    !is.na(Creation_Date) ~ Creation_Date %>%
      substr(nchar(Creation_Date) - 1, nchar(Creation_Date) ) %>%
      as.numeric()
  ))
df = df %>% mutate(Creation_Date = Creation_Date )
#accumulate date
df <- df %>% mutate(Creation_Date_New = Creation_Year+Creation_Month+Creation_Day  )
df <- df %>% select(-Creation_Year, -Creation_Month, -Creation_Day)

#Release_Date
#year
df = df %>%
  mutate(Release_Year = case_when(
    !is.na(Release_Date) ~ Release_Date %>%
      substr(nchar(Release_Date) - 9, nchar(Release_Date)-6 ) %>%
      as.numeric()
  ))
df = df %>% mutate(Release_Year = (Release_Year-2020)* 365 )
#month
df = df %>%
  mutate(Release_Month = case_when(
    !is.na(Release_Date) ~ Release_Date %>%
      substr(nchar(Release_Date) - 4, nchar(Release_Date)-3 ) %>%
      as.numeric()
  ))
df = df %>% mutate(Release_Month = ( Release_Month-1)* 30 )
#day
df = df %>%
  mutate(Release_Day = case_when(
    !is.na(Release_Date) ~ Release_Date %>%
      substr(nchar(Release_Date) - 1, nchar(Release_Date) ) %>%
      as.numeric()
  ))
df = df %>% mutate(Release_Date = Release_Date )
#accumulate date
df <- df %>% mutate(Release_Date_New = Release_Year+Release_Month+Release_Day  )
df <- df %>% select(-Release_Year, -Release_Month, -Release_Day)

#Difference between release and creation data
df <- df %>% mutate(Diff_Date = Release_Date_New-Creation_Date_New  )

#df %>% dfSummary %>% view

In [10]:
%%R
##############################################################################
### Missing data handling 
#Material_Class  2
df[which(is.na(df$Material_Class)),] 
#which(is.na(df$	Material_Class))
#df[866465:866474,]
#df[866531:866539,]
#not in Test_set, delete
df <- df[-which(is.na(df$Material_Class)),]


#sales order header: 43/1409894
missing1 <- df[which(is.na(df$Creation_Date_New)),] 
#missing1 %>% dfSummary %>% view
#Diff_Date: 3683/1409894
missing2 <- df[which(is.na(df$Diff_Date)),] 
#missing2 %>% dfSummary %>% view
#include test data, so they cannot be deleted

ddf <- df

ddf[which(is.na(ddf$Document_Type)),]$Document_Type <- "Order"  #"Order" accounts for 99%, so...
ddf[which(is.na(ddf$Delivery)),]$Delivery <- "Not relevant"   #"Not relevant" accounts for 93.3%, so...
ddf[which(is.na(ddf$Net_Value.y)),]$Net_Value.y <- 615456    #  615455.6 is average of Net_Value.y, so...
ddf[which(is.na(ddf$Organization)),]$Organization <- 0  #Organization: 0 accounts for 67.7% and 1 is 32.3%, so..
ddf[which(is.na(ddf$Creation_Date_New)),]$Creation_Date_New <- 533 #533.3 is average of Creation_Date_New, so...
ddf[which(is.na(ddf$Release_Date_New)),]$Release_Date_New <- 542#542 is average of Release_Date_New, so...
ddf[which(is.na(ddf$Diff_Date)),]$Diff_Date <- 8     #8.2 is average of Diff_Date, so...

#ddf %>% dfSummary %>% view

#Cost_Center could be further used
#Document_Type, Order accounts for 99%, could be further used.
#Both Net_Values are not handled. Maybe log function

ddf

# A tibble: 1,409,892 x 24
   Customer_ID                                                        Reseller
   <chr>                                                                 <dbl>
 1 0x519277E9B1C951A10E3C688949F69A00C623ADC26AADD09B392379A0EF021ADC        0
 2 0xA211C22E3FBED0029B2634CDFC6AAE790BD34630E8434F1C90D7E8E110111D1D        0
 3 0xA211C22E3FBED0029B2634CDFC6AAE790BD34630E8434F1C90D7E8E110111D1D        0
 4 0xA211C22E3FBED0029B2634CDFC6AAE790BD34630E8434F1C90D7E8E110111D1D        0
 5 0x782677F34EA9A276185073E94F31CDC94363BDF74BF0C89BEC8DA17C76015DF4        0
 6 0x2D063C3AC8F56DD0D042D53F4A54E335A8087DB36744F5E8805A68757235659C       NA
 7 0x939FD771B2BE656BC77158389368D0B4449E1B3A95575DAD197DDDC0E92FE05D        0
 8 0x6EB4E02F951932E496A9E363F6383B3310CF71752F498CE1F4E53359CA4FCD88        0
 9 0x308052E5913F6C805ADCCFF5B27BF83DA730DF1D8148145BF5B0E3CC359D302B        0
10 0x308052E5913F6C805ADCCFF5B27BF83DA730DF1D8148145BF5B0E3CC359D302B        0
   Test_set_id
         <

 4 Order         2021-05-13   Completely processed     2963102 BU_C         
 5 Order         2021-10-14   Completely processed     2963102 BU_A         
 6 Order         2020-12-31   Completely processed       93915 BU_A         
 7 Order         2021-04-27   Completely processed     2963102 BU_C         
 8 Credit memo   NA           Not relevant             1247371 BU_A         
 9 Order         2021-07-28   Completely processed     2963102 BU_A         
10 Order         2021-09-26   Completely processed     2963102 BU_C         
   MATKL Type_SOTP Organization Creator_Binary Creation_Date_New
   <dbl>     <dbl>        <dbl>          <dbl>             <dbl>
 1     0         0            0              1               546
 2     0         0            1              0               498
 3     0         0            1              0               498
 4     0         0            1              0               498
 5     0         0            1              0               649
 6    

In [11]:
%%R
dfdf <- ddf %>% select(-c(Customer_ID, Sales_Order, Material_Code, Creation_Date ,Release_Date))
#dfdf %>% dfSummary %>% view

In [12]:
%%R
#Split Data into Labeled and Unlabeled (Test) Sets
labeled_data = dfdf[is.na(dfdf$Test_set_id),]
test_data = dfdf[!is.na(dfdf$Test_set_id),]

labeled_data %>% nrow()
test_data %>% nrow()


write_csv(x = dfdf, file = "train_test_mixed.csv")
write_csv(x = labeled_data, file = "labeled_data.csv")
write_csv(x = test_data, file = "test_data.csv")


R[write to console]: 
R[write to console]: wrote 2.88MB in  0s, 2.15GB/s
R[write to console]: 
R[write to console]: wrote 89.27MB in  0s, 444.11MB/s
R[write to console]: 
R[write to console]: wrote 92.14MB in  0s, 443.31MB/s
R[write to console]: 
R[write to console]: wrote 95.02MB in  0s, 441.29MB/s
R[write to console]: 
R[write to console]: wrote 97.89MB in  0s, 443.06MB/s
R[write to console]: 
R[write to console]: wrote 100.78MB in  0s, 433.09MB/s
R[write to console]: 
R[write to console]: wrote 103.65MB in  0s, 434.97MB/s
R[write to console]: 
R[write to console]: wrote 106.53MB in  0s, 429.22MB/s
R[write to console]: 
R[write to console]: wrote 109.40MB in  0s, 424.54MB/s
R[write to console]: 
R[write to console]: wrote 112.27MB in  0s, 423.68MB/s
R[write to console]: 
R[write to console]: wrote 115.16MB in  0s, 420.12MB/s
R[write to console]: 
R[write to console]: wrote 118.05MB in  0s, 418.29MB/s
R[write to console]: 
R[write to console]: wrote 120.93MB in  0s, 414.11MB/s
R[write

R[write to console]: 
R[write to console]: 
R[write to console]: wrote 2.26MB in  0s, 2.15GB/s
R[write to console]: 
R[write to console]: wrote 83.32MB in  0s, 409.65MB/s
R[write to console]: 
R[write to console]: wrote 85.58MB in  0s, 406.13MB/s
R[write to console]: 
R[write to console]: wrote 87.83MB in  0s, 399.88MB/s
R[write to console]: 
R[write to console]: wrote 90.08MB in  0s, 396.92MB/s
R[write to console]: 
R[write to console]: wrote 92.33MB in  0s, 394.11MB/s
R[write to console]: 
R[write to console]: wrote 94.58MB in  0s, 393.25MB/s
R[write to console]: 
R[write to console]: wrote 96.82MB in  0s, 391.10MB/s
R[write to console]: 
R[write to console]: wrote 99.07MB in  0s, 388.57MB/s
R[write to console]: 
R[write to console]: wrote 101.33MB in  0s, 387.81MB/s
R[write to console]: 
R[write to console]: wrote 103.59MB in  0s, 385.67MB/s
R[write to console]: 
R[write to console]: wrote 105.85MB in  0s, 384.79MB/s
R[write to console]: 
R[write to console]: wrote 108.10MB in  0s, 

R[write to console]: 
R[write to console]: wrote 44.20MB in  0s, 198.87MB/s
R[write to console]: 
R[write to console]: wrote 44.83MB in  0s, 197.41MB/s
R[write to console]: 
R[write to console]: wrote 45.46MB in  0s, 195.01MB/s
R[write to console]: 
R[write to console]: wrote 46.10MB in  0s, 195.28MB/s
R[write to console]: 
R[write to console]: wrote 46.73MB in  0s, 192.91MB/s
R[write to console]: 
R[write to console]: wrote 47.36MB in  0s, 191.19MB/s
R[write to console]: 
R[write to console]: wrote 48.00MB in  0s, 189.31MB/s
R[write to console]: 
R[write to console]: wrote 48.63MB in  0s, 188.15MB/s
R[write to console]: 
R[write to console]: wrote 49.27MB in  0s, 186.59MB/s
R[write to console]: 
R[write to console]: wrote 49.90MB in  0s, 184.93MB/s
R[write to console]: 
R[write to console]: wrote 50.53MB in  0s, 183.88MB/s
R[write to console]: 
R[write to console]: wrote 51.16MB in  0s, 182.83MB/s
R[write to console]: 
R[write to console]: wrote 51.80MB in  0s, 181.68MB/s
R[write to c

In [13]:
#Python Classifier

In [14]:
!pip install pandas==1.5.2
!pip install scikit-learn==1.2.0
!pip install numpy==1.23.0

  Using cached numpy-1.22.4-cp38-cp38-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Anaconda\\Lib\\site-packages\\~-6py\\.libs\\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



  Using cached numpy-1.23.0-cp38-cp38-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.6.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.0 which is incompatible.


In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from typing import List

C:\Anaconda\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [16]:
labeled_data = pd.read_csv('labeled_data.csv')
labeled_data.tail(5)
labeled = labeled_data
#labeled = labeled_data.drop('Customer_ID',axis = 1)
labeled

,Reseller,Test_set_id,Item_Position,Num_Items,Material_Class,Cost_Center,Net_Value.x,Creator,Document_Type,Delivery,Net_Value.y,Business_Unit,MATKL,Type_SOTP,Organization,Creator_Binary,Creation_Date_New,Release_Date_New,Diff_Date
0,0,NaN,1000,12.0,81,0x29B749B825DAAACA593BE27EB5B936ACCEE82D3040CF...,602656,0x2500D9E578134DB667D820EA3100F880DAF13E408E67...,Order,Not relevant,597357,BU_B,0,0,0,1,546,755,209
1,0,NaN,2000,1.0,343,0x92C3D5C9075099A9B5B97D3AB73F6B91E9574569C586...,1127711,0x5DA587FB7ECCB6F511A468513459B7B4A64BE1751BAD...,Order,Completely processed,2963102,BU_C,0,0,1,0,498,498,0
2,0,NaN,3000,1.0,343,0x92C3D5C9075099A9B5B97D3AB73F6B91E9574569C586...,1129411,0x5DA587FB7ECCB6F511A468513459B7B4A64BE1751BAD...,Order,Completely processed,2963102,BU_C,0,0,1,0,498,498,0
3,0,NaN,1000,12.0,343,0x92C3D5C9075099A9B5B97D3AB73F6B91E9574569C586...,987825,0x5DA587FB7ECCB6F511A468513459B7B4A64BE1751BAD...,Order,Completely processed,2963102,BU_C,0,0,1,0,498,498,0
4,0,NaN,1040,1.0,68,0x622BB2FFB2B722627A7572FA2FF67D8BB3ADB48EE0E2...,2963102,0xFAE7D4BB4FF72593E66A29A9F8A92742B3AD7CCFDCE4...,Order,Completely processed,2963102,BU_A,0,0,1,0,649,649,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104074,0,NaN,0,2.0,227,0x419920BE240B5096878D241AFF47DA2EA700E1D6A9F4...,809291,0x3A131A692E36EA026AE30AC8A40843B0962A69ED72B2...,Order,Not relevant,678514,BU_A,1,0,0,0,721,721,0
1104075,0,NaN,0,1.0,227,0x419920BE240B5096878D241AFF47DA2EA700E1D6A9F4...,1127097,0x3A131A692E36EA026AE30AC8A40843B0962A69ED72B2...,Order,Not relevant,678514,BU_A,1,0,0,0,721,721,0
1104076,0,NaN,0,1.0,291,0x0A5539D6244CBE66F58F7CD6B3BE55BB94324F1A4216...,1035467,0x3A131A692E36EA026AE30AC8A40843B0962A69ED72B2...,Order,Not relevant,678514,BU_C,0,0,0,0,721,721,0
1104077,0,NaN,0,1.0,11,0x0A5539D6244CBE66F58F7CD6B3BE55BB94324F1A4216...,122518,0x3A131A692E36EA026AE30AC8A40843B0962A69ED72B2...,Order,Not relevant,678514,BU_C,0,0,0,0,721,721,0


In [17]:
'''
customers = labeled_data.groupby(['Customer_ID']).count().reset_index().sort_values(by=['Reseller'],ascending=False)
customers_300 = customers[0:1000]
customers_300.tail(3)
'''

"\ncustomers = labeled_data.groupby(['Customer_ID']).count().reset_index().sort_values(by=['Reseller'],ascending=False)\ncustomers_300 = customers[0:1000]\ncustomers_300.tail(3)\n"

In [18]:
'''
Customer_ID_300 = customers_300['Customer_ID'].to_numpy()
#Customer_ID_300
labeled = labeled_data
labeled.loc[~labeled['Customer_ID'].isin(Customer_ID_300),['Customer_ID']] = 'not_specified'
#labeled[~labeled['Customer_ID'].isin(Customer_ID_300)]
labeled.head(3)
'''

"\nCustomer_ID_300 = customers_300['Customer_ID'].to_numpy()\n#Customer_ID_300\nlabeled = labeled_data\nlabeled.loc[~labeled['Customer_ID'].isin(Customer_ID_300),['Customer_ID']] = 'not_specified'\n#labeled[~labeled['Customer_ID'].isin(Customer_ID_300)]\nlabeled.head(3)\n"

In [19]:
#Converting Categoricals into one-hot encoding
'''
for col in type_separator(labeled)["categorical"]:  # 注意看这里对于ctegorical data的处理
                                                   
   # trimmed_col = col.strip().replace(" ", "_")     
   # print_if_verbose(f"[INFO] Col:{col},num_of_unq:{num_unq}, applying 1-HOT encoding.")
   onehot_df = pd.get_dummies(labeled[col])
   # onehot_df = onehot_df.add_prefix(trimmed_col + "_1HOTENC_")
   df = pd.concat((labeled, onehot_df), axis=1)

df.head()
'''
df = pd.get_dummies(labeled)
df.head(10)

,Reseller,Test_set_id,Item_Position,Num_Items,Material_Class,Net_Value.x,Net_Value.y,MATKL,Type_SOTP,Organization,...,Document_Type_Order,Document_Type_Order w/o charge,Document_Type_Returns,Delivery_Completely processed,Delivery_Not relevant,Delivery_Not yet processed,Delivery_Partially processed,Business_Unit_BU_A,Business_Unit_BU_B,Business_Unit_BU_C
0,0,NaN,1000,12.0,81,602656,597357,0,0,0,...,1,0,0,0,1,0,0,0,1,0
1,0,NaN,2000,1.0,343,1127711,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
2,0,NaN,3000,1.0,343,1129411,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
3,0,NaN,1000,12.0,343,987825,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
4,0,NaN,1040,1.0,68,2963102,2963102,0,0,1,...,1,0,0,1,0,0,0,1,0,0
5,0,NaN,7040,1.0,18,2963102,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
6,0,NaN,60004,0.0,178,1247371,1247371,0,0,0,...,0,0,0,0,1,0,0,1,0,0
7,0,NaN,2000,1.0,7,866027,2963102,0,0,1,...,1,0,0,1,0,0,0,1,0,0
8,0,NaN,2000,1.0,288,917861,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1
9,0,NaN,2000,1.0,7,97254,2963102,0,0,1,...,1,0,0,1,0,0,0,0,0,1


In [20]:
#Dropping some columns
drop_col = ['Customer_ID', 'Test_set_id','Creator_Binary']
for col in drop_col:
  if col in list(df.columns):
    df = df.drop(col, axis=1)
df.head(3)

,Reseller,Item_Position,Num_Items,Material_Class,Net_Value.x,Net_Value.y,MATKL,Type_SOTP,Organization,Creation_Date_New,...,Document_Type_Order,Document_Type_Order w/o charge,Document_Type_Returns,Delivery_Completely processed,Delivery_Not relevant,Delivery_Not yet processed,Delivery_Partially processed,Business_Unit_BU_A,Business_Unit_BU_B,Business_Unit_BU_C
0,0,1000,12.0,81,602656,597357,0,0,0,546,...,1,0,0,0,1,0,0,0,1,0
1,0,2000,1.0,343,1127711,2963102,0,0,1,498,...,1,0,0,1,0,0,0,0,0,1
2,0,3000,1.0,343,1129411,2963102,0,0,1,498,...,1,0,0,1,0,0,0,0,0,1


In [21]:
#Features and Labels
labels = df['Reseller']
print(type(labels))
# Remove the labels from the features
# axis 1 refers to the columns
features= df.drop('Reseller', axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
# features = np.array(features)

<class 'pandas.core.series.Series'>


In [22]:
#Training and Testing Set
# Split the data into training and testing sets
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size = 0.25,
                                                                           random_state = 42)
print(train_labels.size)
print(val_labels.value_counts())
print(train_labels.value_counts())
'''
0    206072
1     69948
'''

828059
0    206072
1     69948
Name: Reseller, dtype: int64
0    620012
1    208047
Name: Reseller, dtype: int64


'\n0    206072\n1     69948\n'

In [23]:
# Instantiate model 
rf = RandomForestClassifier(random_state=0) #parametres to be tuned

# Train the model on training data
rf.fit(train_features, train_labels)

train_preds = rf.predict(train_features)
val_preds = rf.predict(val_features)

In [24]:
val_preds.mean()

0.2246467647271937

In [25]:
val_labels.mean()

0.25341641910006524

In [26]:
#Metrics
score = balanced_accuracy_score(val_labels, val_preds)
print('score_1', score)
# above are official balanced_accuracy score

def balanced_accuracy(a): #输入是一个2*2矩阵
   sum_0 = np.sum(a, axis=0)
   # sum_1= np.sum(a, axis=1)
   sensitivity = a[0][0] / sum_0[0]
   specificity = a[1][1] / sum_0[1]
   balanced_accuracy = (sensitivity + specificity) / 2
   return balanced_accuracy

from sklearn.metrics import confusion_matrix
c_mat = confusion_matrix(val_labels, val_preds) # 我自己写的metrics
score_2 = balanced_accuracy(c_mat)
print('score_2', score_2)                      
'''
score_1 0.9991754035923639
score_2 0.9996949075129307 ,可见metrics 本身并没有问题。
'''
def evaluate(y, y_hat , model_name):
   # use balanced accuracy metric
    cm = confusion_matrix(y, y_hat)
    print('*'*20)
    print('Confusion Matrix for model {0}: '.format(model_name))
    print(cm)
    sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])  
    specificity = cm[1,1]/(cm[1,0]+cm[1,1])
    bac = (sensitivity+specificity)/2
    print('Specifity of model {0}: '.format(model_name))
    print(specificity)
    print('Sensitivity of model {0}: '.format(model_name))
    print(sensitivity)
    print('Balanced accuracy of model {0}: '.format(model_name))
    print(bac)
    print('*'*20)

score_1 0.8966947420112207
score_2 0.9308934145584531


In [27]:
val_preds_df = pd.DataFrame()
val_preds_df["val"] = pd.Series(val_preds)
val_preds_df[val_preds_df[['val']].isnull().T.any()]

,val
